# 제 9 장 __인공신경망의 이해와 활용__ (2)
___

## __사전설정__
---

(1) 저장소 데이터 가져오기

In [ ]:
!rm -rf /content/BizDataAnalysis/

In [ ]:
!git clone https://github.com/BizStat/BizDataAnalysis.git


(2) matplotlib 환경에서 한글 사용

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -f -v
!rm ~/.cache/matplotlib -rf

런타임 메뉴에서 '세션 다시 시작' 후 다음의 명령문 실행

In [ ]:
from matplotlib import rc
rc('font', family='NanumGothicCoding')
rc('axes', unicode_minus=False)

(3) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

___

## 9.1 __비지도 학습을 위한 인공신경망 모형__

### (1) 오토인코더 사례

__1.  데이터 전처리__

* Red Wine Quality 데이터 가져오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
wine = pd.read_table('/content/BizDataAnalysis/DATA/winequality-red.csv',sep=',')

In [ ]:
wine.head()

* 분석 데이터의 추출과 표준화

In [ ]:
X = wine.iloc[:,0:-1]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

In [ ]:
Xt = scaler.transform(X)

__2.  분석에 사용할 신경망 모형 설정__

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
encoder = Sequential([Dense(2,input_dim=11)])
decoder = Sequential([Dense(11)])

In [ ]:
wine_ae_mod = Sequential([encoder, decoder],name='Wine_AE_Model')

In [ ]:
wine_ae_mod.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(wine_ae_mod, show_shapes=True)

__3. 오토인코더 모형 컴파일__

In [ ]:
wine_ae_mod.compile(optimizer='adam', loss='mse',metrics=['mse'])

__4.  오토인코더 모형 학습__

In [ ]:
history = wine_ae_mod.fit(
  Xt, Xt,
  validation_split = 0.2,
  epochs=100
)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.plot(hist['epoch'], hist['mse'],label='Train Error')
plt.plot(hist['epoch'], hist['val_mse'],label = 'Test Error')
#  plt.ylim([0,5])
plt.legend()
plt.show()


__5.  오토인코더 모형의 예측결과 살펴보기__

* 신경망을 구성하는 특정 층의 가중치 출력

In [ ]:
encoder.get_weights()

In [ ]:
en_parm = encoder.get_weights()[0]

In [ ]:
plt.scatter(en_parm[:,0], en_parm[:,1])
plt.title('인코더의 연결망 가중값')
plt.xlabel('주성분 1')
plt.ylabel('주성분 2')
plt.show()

* 신경망을 구성하는 특정 층의 예측값 출력

In [ ]:
encoder.predict(Xt)

In [ ]:
ae_res = encoder.predict(Xt)

In [ ]:
plt.scatter(ae_res[:,0], ae_res[:,1])
plt.title('축소된 데이터의 산포도')
plt.xlabel('주성분 1')
plt.ylabel('주성분 2')
plt.show()

In [ ]:
#@title PCA 결과 시각화를 위한 biplot 함수 (출처: https://sukhbinder.wordpress.com/2015/08/05/biplot-with-python/)
def biplot(score,coeff,pcax,pcay,labels=None):
  pca1=pcax-1
  pca2=pcay-1
  xs = score[:,pca1]
  ys = score[:,pca2]
  n=coeff.shape[0] # 수정
  scalex = 1.0/(xs.max()- xs.min())
  scaley = 1.0/(ys.max()- ys.min())
  plt.scatter(xs*scalex,ys*scaley)
  for i in range(n):
    plt.arrow(0, 0, coeff[i,pca1], coeff[i,pca2],color='r',alpha=0.5)
    if labels is None:
      plt.text(coeff[i,pca1]* 1.15, coeff[i,pca2] * 1.15, "Var"+str(i+1), color='g', ha='center', va='center')
    else:
      plt.text(coeff[i,pca1]* 1.15, coeff[i,pca2] * 1.15, labels[i], color='black', ha='center', va='center') # 글자색 수정
  plt.xlim(-1,1)
  plt.ylim(-1,1)
  plt.xlabel("주성분{}".format(pcax))
  plt.ylabel("주성분{}".format(pcay))
  plt.grid()

In [ ]:
biplot(ae_res,en_parm,1,2,labels=scaler.feature_names_in_)

---

## 9.2 __순차적 데이터 분석을 위한 인공 신경망 모형__

### (1) 순환신경망을 이용한 삼성전자 주가 예측

__1.  데이터 전처리__

* 데이터 가져오기

In [ ]:
sec = pd.read_table('/content/BizDataAnalysis/DATA/삼성전자일일종가.csv',sep=',')

* 데이터 살펴보기

In [ ]:
sec.head()

In [ ]:
sec.info()

In [ ]:
# * 시간 데이터의 형식 변경
sec['DATE'] = pd.to_datetime(sec['yyyymmdd'], format='%Y-%m-%d')

# * 날짜를 인덱스로 설정
sec.set_index('DATE', inplace=True)

# * '종가'만 추출
price_data = sec['price']

# * 종가 시계열 그래프
plt.figure(figsize=(12, 6))
plt.plot(price_data)
plt.title('2025년 삼성전자 주가')
plt.xlabel('날짜')
plt.ylabel('종가')
# plt.grid(True)
plt.show()

* 입력층과 출력층 분리

In [ ]:
tmp = sec.iloc[:,1].values.tolist()

In [ ]:
win_size = 5
X = []
Y = []
for i in range(len(tmp) - win_size):
    X.append(tmp[i:i+win_size])
    Y.append(tmp[i+win_size])

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
X[0]

In [ ]:
Y[0]

In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
Xt = np.array(X)
Yt = np.array(Y)

__2.  분석에 사용할 신경망 모형 설정__

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [ ]:
sec_model = Sequential([
    LSTM(15, activation='relu', input_shape=(win_size,1)),
    Dense(1)
])

In [ ]:
sec_model.summary()

__3. 모형의 컴파일과 학습__

In [ ]:
sec_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [ ]:
history = sec_model.fit(
  Xt, Yt,
  validation_split = 0.2,
  epochs=200,
  batch_size=20,
  verbose=1
)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.plot(hist['epoch'], hist['mse'],label='Train Error')
plt.plot(hist['epoch'], hist['val_mse'],label = 'Test Error')
#  plt.ylim([0,5])
plt.legend()
plt.show()


__4. 주가 예측 결과__

In [ ]:
pred_y = sec_model.predict(Xt)

In [ ]:
plt.figure()
plt.plot(Yt, color='red', label='실제값')
plt.plot(pred_y, color='blue', label='예측값')
plt.title('삼성전자 주가 예측 결과')
plt.xlabel('날짜')
plt.ylabel('가격')
plt.legend()
plt.show()

*